# **Projeto: Previsão de Intenção de Compra de Clientes em Loja Web**

**Descrição do Projeto**

Neste projeto, nosso objetivo é criar um sistema inteligente para antecipar a intenção de compra dos clientes em um site de e-commerce. Imagine poder prever quais clientes têm maior probabilidade de realizar compras online, com base em suas características e comportamentos passados. Essa capacidade de prever a intenção de compra não só aprimorará a experiência do cliente, mas também permitirá que a empresa direcione seus esforços de marketing de forma mais eficaz.

**Objetivo**

Queremos desenvolver um modelo preditivo capaz de analisar os padrões de comportamento dos clientes e identificar sinais que indicam a propensão deles para realizar compras no site da empresa. Para isso, vamos usar uma base de dados que contém informações detalhadas sobre os clientes, incluindo:

Dados demográficos (idade, renda, etc.)

Informações sobre compras anteriores



# Base de dados:

Year_Birth: Ano de nascimento do cliente.

Education: Nível de escolaridade do cliente.

Marital_Status: Estado civil do cliente.

Income: Renda anual da família do cliente.

Kidhome: Número de crianças na casa do cliente.

Recency: Número de dias desde a última compra do cliente.

Complain: 1 se o cliente reclamou nos últimos 2 anos, 0 caso contrário.

MntWines: Valor gasto em vinhos nos últimos 2 anos.

MntFruits: Valor gasto em frutas nos últimos 2 anos.

MntMeatProducts: Valor gasto em carnes nos últimos 2 anos.

MntFishProducts: Valor gasto em peixes nos últimos 2 anos.

MntSweetProducts: Valor gasto em doces nos últimos 2 anos.

MntGoldProds: Valor gasto em produtos de ouro nos últimos 2 anos.

NumDealsPurchases: Número de compras feitas com desconto

NumStorePurchases: Número de compras feitas diretamente nas lojas.

NumWebVisitsMonth: Número de visitas ao site da empresa no último mês.






**WebPurchases: Número de compras feitas pelo site da empresa.**

# ETAPA 1:

**Preparação dos Dados**

**Exploração e Limpeza:** Analisar e limpar os dados para garantir que estejam prontos para a modelagem.

**Análise:** Construa uma storytelling com gráficos, analisando e retirando insights das informações.

In [164]:
# Importando as bibliotecas necessarias 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 


Após importar as bibliotecas necessárias, carregamos nosso arquivo CSV e o transformamos em um DataFrame, o que nos permite utilizar o pandas para realizar toda a manipulação dos dados.

In [166]:
# Carregndo a base de dados 
df = pd.read_csv('marketing_campaign.csv', delimiter=';')

In [167]:
# Verificando as primeiras linhas do DataFrame 
df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumStorePurchases,NumWebVisitsMonth,Complain,WebPurchases
0,1957,Graduation,Single,58138.0,0,58,635,88,546,172,88,88,4,7,0,1
1,1954,Graduation,Single,46344.0,1,38,11,1,6,2,1,6,2,5,0,0
2,1965,Graduation,Together,71613.0,0,26,426,49,127,111,21,42,10,4,0,1
3,1984,Graduation,Together,26646.0,1,26,11,4,20,10,3,5,4,6,0,0
4,1981,PhD,Married,58293.0,1,94,173,43,118,46,27,15,6,5,0,1


### Análise Inicial
Podemos observar que a base de dados é composta majoritariamente por colunas numéricas, o que facilita a manipulação e o uso em modelos de machine learning. Temos apenas algumas colunas categóricas, como Education e Marital_Status, que poderão ser transformadas posteriormente para variáveis numéricas através de técnicas como label encoding ou one-hot encoding.

In [169]:
# Visualizando os valores unicos nas colunas categoricas
df['Education'].unique()
df['Marital_Status'].unique()

array(['Single', 'Together', 'Married', 'Divorced', 'Widow', 'Alone',
       'Absurd', 'YOLO'], dtype=object)

In [170]:
# Verifica quantas vezes os valores inconsistentes aparecem ANTES da substituição
valores_inconsistentes = ['Alone', 'Absurd', 'YOLO']
df['Marital_Status'].value_counts().loc[valores_inconsistentes]


Marital_Status
Alone     3
Absurd    2
YOLO      2
Name: count, dtype: int64

In [228]:
# Criando um dicionário de mapeamento específico para esses valores
marital_map = {
    'Single': 'Single',
    'Together': 'Together',
    'Married': 'Married',
    'Widow': 'Widow',
    'Alone': 'Single',
    'Absurd': 'Single',
    'YOLO': 'Single'
}


# Aplicando o mapeamento
df['Marital_Status'] = df['Marital_Status'].map(marital_map).fillna(df['Marital_Status'])
df['Marital_Status'].value_counts()

Marital_Status
Married     864
Together    580
Single      487
Widow        77
Name: count, dtype: int64

### Tratamento dos valores inconsistentes na coluna Marital_Status
Durante a análise inicial dos valores únicos da coluna Marital_Status, identificamos algumas categorias inconsistentes ou fora do padrão: 'Alone', 'Absurd' e 'YOLO'.

Essas categorias representam um número pequeno de registros (3 para 'Alone' e 2 para cada uma das outras duas) e não correspondem a status civis convencionais.

Para manter a consistência dos dados e facilitar o treinamento do modelo, optamos por agrupar essas categorias como 'Single' (solteiro), que é a categoria mais próxima em significado para 'Alone', e uma boa aproximação para as outras duas.

Esse procedimento ajuda a:

- Reduzir a complexidade das categorias da variável;

- Evitar possíveis ruídos que valores incomuns podem causar no modelo;

- Garantir que todos os registros sejam classificados em categorias válidas.

In [231]:
#  Verificando as Informações Básicas do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year_Birth         2240 non-null   int64  
 1   Education          2240 non-null   object 
 2   Marital_Status     2008 non-null   object 
 3   Income             2240 non-null   float64
 4   Kidhome            2240 non-null   int64  
 5   Recency            2240 non-null   int64  
 6   MntWines           2240 non-null   int64  
 7   MntFruits          2240 non-null   int64  
 8   MntMeatProducts    2240 non-null   int64  
 9   MntFishProducts    2240 non-null   int64  
 10  MntSweetProducts   2240 non-null   int64  
 11  MntGoldProds       2240 non-null   int64  
 12  NumStorePurchases  2240 non-null   int64  
 13  NumWebVisitsMonth  2240 non-null   int64  
 14  Complain           2240 non-null   int64  
 15  WebPurchases       2240 non-null   int64  
dtypes: float64(1), int64(13)

### Análise
Aqui confirmamos que o dataset possui 2240 registros e que a maioria das colunas está completa. As únicas colunas categóricas são Education e Marital_Status, enquanto as demais são numéricas.

A coluna Year_Birth, embora esteja como inteiro, pode ser convertida em idade ou em uma data para gerar novos atributos úteis, como faixa etária, idade média por perfil, entre outros.

In [234]:
# Verificando valores nulos
df.isnull().sum()

Year_Birth             0
Education              0
Marital_Status       232
Income                 0
Kidhome                0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumStorePurchases      0
NumWebVisitsMonth      0
Complain               0
WebPurchases           0
dtype: int64

### Análise
Identificamos 24 valores nulos apenas na coluna Income (renda anual). Como eles representam uma quantidade muito pequena de registros em relação ao total do DataFrame, uma boa prática é substituir esses valores pela média da coluna, garantindo a integridade do conjunto de dados sem perda de informação.

In [178]:
df['Income'].fillna(df['Income'].mean(), inplace=True)

df.isnull().sum()

C:\Users\Jonathan\AppData\Local\Temp\ipykernel_21012\1562807081.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Income'].fillna(df['Income'].mean(), inplace=True)


Year_Birth             0
Education              0
Marital_Status       232
Income                 0
Kidhome                0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumStorePurchases      0
NumWebVisitsMonth      0
Complain               0
WebPurchases           0
dtype: int64

In [179]:
# Visualizar estátisticas básica das colunas númericas 
df.describe()

,Year_Birth,Income,Kidhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumStorePurchases,NumWebVisitsMonth,Complain,WebPurchases
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,1968.805804,52247.251354,0.444196,49.109375,303.935714,26.302232,166.950000,37.525446,27.062946,44.021875,5.790179,5.316518,0.009375,0.503571
std,11.984069,25037.797168,0.538398,28.962453,336.597393,39.773434,225.715373,54.628979,41.280498,52.167439,3.250958,2.426645,0.096391,0.500099
min,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1959.000000,35538.750000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,1.000000,9.000000,3.000000,3.000000,0.000000,0.000000
50%,1970.000000,51741.500000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,8.000000,24.000000,5.000000,6.000000,0.000000,1.000000
75%,1977.000000,68289.750000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,33.000000,56.000000,8.000000,7.000000,0.000000,1.000000
max,1996.000000,666666.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,263.000000,362.000000,13.000000,20.000000,1.000000,1.000000


### Análise 

com a descrição estatisticas do nosso DataFrame podemos visualizar que 

In [181]:
'''# Visualizando a distribuição das colunas númericas 
numeric_cols = df.select_dtypes(include=['number']).columns

# Definindo o número de fráficos por linha para organizar a visualização
n_cols = 3 # 3 Gráficos por linha
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols # Calcula o número de linhas necessário

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Ajusta o tamanho da figura

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df[col], kde=True, bins=15)
    plt.title(f'Distribuição de {col}')
    plt.xlabel(col)
    plt.ylabel('Frequeência')
    plt.tight_layout()
plt.show()'''

"# Visualizando a distribuição das colunas númericas \nnumeric_cols = df.select_dtypes(include=['number']).columns\n\n# Definindo o número de fráficos por linha para organizar a visualização\nn_cols = 3 # 3 Gráficos por linha\nn_rows = (len(numeric_cols) + n_cols - 1) // n_cols # Calcula o número de linhas necessário\n\nplt.figure(figsize=(n_cols * 5, n_rows * 4)) # Ajusta o tamanho da figura\n\nfor i, col in enumerate(numeric_cols):\n    plt.subplot(n_rows, n_cols, i + 1)\n    sns.histplot(df[col], kde=True, bins=15)\n    plt.title(f'Distribuição de {col}')\n    plt.xlabel(col)\n    plt.ylabel('Frequeência')\n    plt.tight_layout()\nplt.show()"

In [183]:
'''# Selecionando apenas as colunas categóricas 
categorical_cols = df.select_dtypes(include=['object']).columns

# Itera sobre cada coluna categórica e craiar um gráfico de barras separdo
for col in categorical_cols:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=col, data=df, palette='viridis', order=df[col].value_counts().index, hue=col, legend=False)
    plt.title(f'Contagem de {col}')
    plt.xlabel(col)
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()'''

"# Selecionando apenas as colunas categóricas \ncategorical_cols = df.select_dtypes(include=['object']).columns\n\n# Itera sobre cada coluna categórica e craiar um gráfico de barras separdo\nfor col in categorical_cols:\n    plt.figure(figsize=(8, 6))\n    sns.countplot(x=col, data=df, palette='viridis', order=df[col].value_counts().index, hue=col, legend=False)\n    plt.title(f'Contagem de {col}')\n    plt.xlabel(col)\n    plt.ylabel('Contagem')\n    plt.xticks(rotation=45, ha='right')\n    plt.tight_layout()\n    plt.show()"

In [184]:
'''# Analisando apenas as colunas numericas 
numeric_df = df[numeric_cols]

plt.figure(figsize=(12, 10))
sns.heatmap(numeric_df.corr(),
            annot=True,
            cmap='coolwarm',
            fmt='.2f',
            linewidths=.5)
plt.title('Matriz de Correlação', fontsize=16)
plt.show()'''

"# Analisando apenas as colunas numericas \nnumeric_df = df[numeric_cols]\n\nplt.figure(figsize=(12, 10))\nsns.heatmap(numeric_df.corr(),\n            annot=True,\n            cmap='coolwarm',\n            fmt='.2f',\n            linewidths=.5)\nplt.title('Matriz de Correlação', fontsize=16)\nplt.show()"

# ETAPA 2:
**Pré-processamento**

**Análise Correlação:** Verifique a correlação entre as váriaveis e análise se há espaço para retirar váriaveis que não te parecem importantes.

**Codificação de Variáveis Categóricas:** Transformar variáveis categóricas em um formato que os modelos de machine learning possam interpretar.


**Separe a base em Y, X e Treino e teste:**: Faça a separação da base.

**Realize a padronização dos dados**: Padronize os dados para garantir eficiência no modelo e eficácia.








In [187]:
# seu código aqui

# ETAPA 3:

**Modelagem**

Escolha ao menos 2 técnicas de machine learning e rode 2 modelos, afim de identificar qual tem o melhor resultado para essa base. Lembrando que estamos lidando com uma classificação binária.

In [189]:
# seu código aqui

# ETAPA 4:

**Avaliação**

Avalie os resultados encontrados nos dois modelos e identifique qual te pareceu realizar melhor as previsões.

Utilize além das métricas padrões a matriz de confusão.

In [191]:
# seu código aqui